In [1]:
# Julia 1.11.6
using Combinatorics
using IterativeSolvers  # for LSQR
using SparseArrays     # Required for SparseMatrixCSC in Julia 1.0+
using LinearAlgebra    # For triu function
const IntMat = SparseMatrixCSC{Float64,Int64}
const FloatMat = SparseMatrixCSC{Float64,Int64}


SparseMatrixCSC{Float64, Int64}

In [2]:
# spones has been removed by Julia 1.11.6
function spones(A::SparseMatrixCSC)
    I, J, V = findnz(A)
    return sparse(I, J, ones(eltype(A), length(V)), size(A)...)
end

spones (generic function with 1 method)

In [3]:
# function to read .paj files (THANKS BRAD NELSON for writing this!)
function read_paj_sparse(fname::AbstractString)
    n = 0
    local A, I, J, V
    local labels
    open(fname) do f
        # get number of vertices                                                                                                                                                     
        while true
            line = readline(f)
            if length(line) > 8 && line[1:9] == "*vertices"
                n = parse(Int64, line[10:end])
                break
            end
        end
        # get labels                                                                                                                                                                 
        labels = Array{String}(undef, n)
        while true
            line = readline(f)
            if length(line) >=9 && line[1:9] == "*vertices"; break; end
        end
        for i = 1:n
            line = readline(f)
            labels[i] = line[7:end-1]
        end
        # get edges                                                                                                                                                                  
        I, J, V = Int64[], Int64[], Float64[]
        while true
            line = readline(f)
            if length(line) > 4 && line[1:5] == "*arcs"; break; end
        end
        while true
            line = readline(f)
            if length(line) < 2; break; end
            (is, js, vs) = split(line)
            push!(I, parse(Int64, is))
            push!(J, parse(Int64, js))
            push!(V, parse(Float64, vs))
        end
    end
    A = convert(FloatMat, sparse(I, J, V, n, n))
    return (A, labels)
end


read_paj_sparse (generic function with 1 method)

In [4]:
# gradient matrix from graph adjacency matrix
# returns gradient matrix and map of edges to indices 1, 2, ..., #edges
function grad_mat(A::FloatMat)
    edge_map = Dict{NTuple{2, Int64}, Int64}()
    I, J, V = Int64[], Int64[], Int64[]
    curr_edge_ind = 1
    num_vertices = size(A, 2)
    for j in 1:num_vertices, i in findall(!iszero, A[:,j])
        # Force alternating function by ordering edges
        if i < j
            edge_map[(i, j)] = curr_edge_ind
            push!(I, curr_edge_ind, curr_edge_ind)
            push!(J, i, j)
            push!(V, -1, 1)
            curr_edge_ind += 1
        end
    end
    grad = convert(FloatMat, sparse(I, J, V, length(edge_map), num_vertices))
    return (grad, edge_map)
end

grad_mat (generic function with 1 method)

In [5]:
# curl matrix from graph adjacency matrix
# edge_map takes (i, j) --> index 1, 2, ..., #edges
function curl_mat(A::FloatMat, edge_map::Dict{NTuple{2, Int64}, Int64})
    num_vertices = size(A, 2)
    # ordering of nodes by degree
    deg_order = zeros(Int64, num_vertices)
    deg_order[sortperm(vec(sum(spones(A), dims=1)))] = collect(1:num_vertices)
    tri_map = Dict{NTuple{3, Int64}, Int64}()
    curr_tri_ind = 1
    I, J, V = Int64[], Int64[], Int64[]
    for i in 1:num_vertices
        pos = deg_order[i]
        # Keep neighbors with larger degree
        neighbors = filter(v -> deg_order[v] > pos, findall(!iszero, A[:, i]))
        for (j, k) in combinations(neighbors, 2)
            if A[j, k] > 0.0
                # triangle {i, j, k}
                a, b, c = sort([i, j, k])
                tri_map[(a, b, c)] = curr_tri_ind
                push!(I, curr_tri_ind, curr_tri_ind, curr_tri_ind)
                push!(J, edge_map[(a, b)], edge_map[(a, c)], edge_map[(b, c)])
                push!(V, 1, -1, 1)
                curr_tri_ind += 1
            end
        end
    end
    curl = convert(FloatMat, sparse(I, J, V, length(tri_map), length(edge_map)))
    return (curl, tri_map)
end

curl_mat (generic function with 1 method)

In [6]:
# Hodge decomposition for an edge flow on a graph
# returns node potential f, curl component Φ, and Harmonic component X_H
# such that X = gradf + X_H + curl'Φ
function hodge_decomp(X::Vector{Float64}, grad::FloatMat, curl::FloatMat)
    @assert size(grad, 1) == length(X) == size(curl, 2)
    f = lsqr(grad, X; atol=1e-10, btol=1e-10, conlim=1e12)
    Φ = lsqr(curl', X; atol=1e-10, btol=1e-10, conlim=1e12)
    X_H = X - curl' * Φ - grad * f
    return (f, Φ, X_H)
end

hodge_decomp (generic function with 1 method)

In [7]:
# read data
@time A, labels = read_paj_sparse("Florida.paj")

# form gradient and curl matrices
Asym = max.(A, A') # symmetrize A to be undirected graph
@time (grad, edge_map) = grad_mat(Asym)
@time (curl, tri_map) = curl_mat(Asym, edge_map)

# sanity check: make sure A * B = 0
curlgrad = curl * grad
println("min(curl grad) = $(minimum(curlgrad)), max(curl grad) = $(maximum(curlgrad))")

  0.085907 seconds (348.23 k allocations: 18.870 MiB, 3.51% gc time, 97.61% compilation time)
  0.015451 seconds (6.97 k allocations: 1.331 MiB, 98.94% compilation time)
  0.001662 seconds (73.61 k allocations: 6.682 MiB)
min(curl grad) = 0.0, max(curl grad) = 0.0


In [8]:
# form an edge from upper triangular part of matrix and ordering of edges
function edge_flow(T::FloatMat, edge_map::Dict{NTuple{2, Int64}, Int64})
    X = zeros(length(edge_map))
    B = triu(T, 1)
    for j in 1:size(B, 2), i in findall(!iszero, B[:,j])
        X[edge_map[(i, j)]] = B[i, j]
    end
    return X
end

@time X = edge_flow(A - A', edge_map) # takes asymmetry into account

# decompose edge flow 
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)

# sanity check: are our components orthogonal?
x1 = grad * f
x2 = curl' * Φ
@show x1' * x2, x1' * X_H, x2' * X_H


  0.108612 seconds (301.50 k allocations: 16.062 MiB, 2.29% gc time, 99.88% compilation time)
  0.236136 seconds (424.73 k allocations: 42.667 MiB, 11.67% gc time, 96.70% compilation time)
(x1' * x2, x1' * X_H, x2' * X_H) = (6.547712976453023e-13, 2.367350969452149e-9, 5.977096864765869e-7)


(6.547712976453023e-13, 2.367350969452149e-9, 5.977096864765869e-7)

In [9]:
# get potential function ranking
ranking = labels[sortperm(f, rev=true)]
@show ranking[1:5]
@show ranking[end-4:end]

# how much of norm due to gradient potential?
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

ranking[1:5] = ["Respiration", "Mackerel", "Predatory Crabs", "Brotalus", "Lizardfish"]
ranking[end - 4:end] = ["Oscillatoria", "Drift Algae", "DOC", "Roots", "Input"]
frac_potential = 0.10523817822463569


0.10523817822463569

In [10]:
# only consider sign of edge
X = edge_flow(sign.(A - A'), edge_map)
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)
# get potential function ranking
ranking = labels[sortperm(f, rev=true)]
@show ranking[1:5]
@show ranking[end-4:end]
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

  0.008895 seconds (1.48 k allocations: 21.550 MiB, 18.92% gc time)
ranking[1:5] = ["Raptors", "Output", "Crocodiles", "Respiration", "Dolphin"]
ranking[end - 4:end] = ["Synedococcus", "2um Spherical Phytoplankt", "Benthic Phytoplankton", "Small Diatoms (<20um)", "Input"]
frac_potential = 0.7872575994111707


0.7872575994111707

In [11]:
# only consider asymmetric flows
A1 = spones(A)
B = A1 .* A1' # bi-directional flows
U = A1 - B
X = edge_flow(sign.(U - U'), edge_map)
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)

# get potential function ranking
ranking = labels[sortperm(f, rev=true)]
@show ranking[1:5]
@show ranking[end-4:end]
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

  0.008015 seconds (1.48 k allocations: 21.550 MiB, 15.05% gc time)
ranking[1:5] = ["Raptors", "Output", "Crocodiles", "Respiration", "Dolphin"]
ranking[end - 4:end] = ["2um Spherical Phytoplankt", "Synedococcus", "Benthic Phytoplankton", "Small Diatoms (<20um)", "Input"]
frac_potential = 0.7958274288086641


0.7958274288086641

In [12]:
# find inconsistent triangles
ordered_tris = sort(collect(keys(tri_map)), by=(k -> Φ[tri_map[k]]), rev=true)
for i = 1:5
    @show labels[[ordered_tris[i]...]]
end


labels[[ordered_tris[i]...]] = ["Dinoflagellates", "Meroplankton", "Input"]
labels[[ordered_tris[i]...]] = ["Meroplankton", "Water POC", "Input"]
labels[[ordered_tris[i]...]] = ["Epiphytes", "Thor Floridanus", "Water POC"]
labels[[ordered_tris[i]...]] = ["Oscillatoria", "Echinoderma", "Parrotfish"]
labels[[ordered_tris[i]...]] = ["Coral", "Parrotfish", "Respiration"]


In [13]:
pf = findfirst(x -> x == "Parrotfish", labels)
@show labels[findall(!iszero, A[:,pf])]
@show labels[findall(!iszero, A[pf,:])]


labels[findall(!iszero, A[:, pf])] = ["Synedococcus", "Oscillatoria", "Drift Algae", "Epiphytes", "Meiofauna", "Sponges", "Coral", "Echinoderma", "Bivalves", "Detritivorous Polychaetes", "Predatory Polychaetes", "Suspension Feeding Polych", "Macrobenthos"]
labels[findall(!iszero, A[pf, :])] = ["Tarpon", "Barracuda", "Other Pelagic Fishes", "Loon", "Greeb", "Pelican", "Comorant", "Predatory Ducks", "Raptors", "Crocodiles", "Dolphin", "Water POC", "Output", "Respiration"]


14-element Vector{String}:
 "Tarpon"
 "Barracuda"
 "Other Pelagic Fishes"
 "Loon"
 "Greeb"
 "Pelican"
 "Comorant"
 "Predatory Ducks"
 "Raptors"
 "Crocodiles"
 "Dolphin"
 "Water POC"
 "Output"
 "Respiration"

In [14]:
A, labels = read_paj_sparse("Michigan.paj")
Asym = max.(A, A') # symmetrize A to be undirected graph
@time (grad, edge_map) = grad_mat(Asym)
@time (curl, tri_map) = curl_mat(Asym, edge_map)

# only consider asymmetric flows
A1 = spones(A)
B = A1 .* A1' # bi-directional flows
U = A1 - B
X = edge_flow(sign.(U - U'), edge_map)
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)

# get potential function ranking
ranking = labels[sortperm(f, rev=true)]
@show ranking[1:5]
@show ranking[end-4:end]
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential


  0.000028 seconds (297 allocations: 98.812 KiB)
  0.000072 seconds (2.97 k allocations: 322.781 KiB)
  0.000154 seconds (711 allocations: 323.016 KiB)
ranking[1:5] = ["Output", "Respiration", "Sea lamprey", "Detritus", "Juv. Lake Trout"]
ranking[end - 4:end] = ["Calanoids", "Flagellates", "Blue-greenGree", "Diatoms", "Input"]
frac_potential = 0.7025775654157067


0.7025775654157067